I. IMPORT LIBRARY

In [0]:
from pyspark.sql.types import StructField, StructType, IntegerType, StringType, BooleanType, DateType, DecimalType
from pyspark.sql.functions import col, when, sum, avg, row_number, expr, create_map, lit
from pyspark.sql.window import Window
from pyspark.sql import SparkSession 
from itertools import chain
import re

In [0]:
# Initialize Spark Session
spark = SparkSession.builder.appName("SilverLayerShopping").getOrCreate()

II. LOAD DATA INTO SILVER LAYER

1. MISSING AND DUPLICATE DATA CLEANING

In [0]:
bronze_shopping_df = spark.read.format("delta").table("brian_work.bronze_layer.bronze_shopping_data")

In [0]:
silver_shopping_df=bronze_shopping_df

In [0]:
display(silver_shopping_df.describe())

In [0]:
# Check null data
display(silver_shopping_df.select([
    (silver_shopping_df[c].isNull().cast("int")).alias(c) for c in silver_shopping_df.columns
]).groupBy().sum())

In [0]:
#Check duplicate value
duplicates_df = silver_shopping_df.groupBy(silver_shopping_df.columns)\
    .count()\
    .filter(col("count") > 1)
display(duplicates_df)


The dataset show that there is no null value in the whole dataset, describe function also reflect that we do not have any 0 value across 3900 rows, no duplicated row recorded, so this is the clean dataset. We can continue check data type and categorize it if we can

In [0]:
display(silver_shopping_df.limit(12))

2. FEATURE ENGINEERING AND DATA TRANSFORMATION

In [0]:
silver_shopping_df = silver_shopping_df.withColumn(
    "Subscription_Status", when(silver_shopping_df["Subscription_Status"] == "Yes", 1).otherwise(0)
).withColumn(
    "Discount_Applied", when(silver_shopping_df["Discount_Applied"] == "Yes", 1).otherwise(0)
).withColumn(
    "Promo_Code_Used", when(silver_shopping_df["Promo_Code_Used"] == "Yes", 1).otherwise(0)
)

In [0]:
# Define frequency mapping dictionary
frequency_mapping = {
    "Daily": 365, "Weekly": 52, "Fortnightly": 26, "Monthly": 12, "Annually": 1, "Quarterly": 4, "Bi-Weekly": 104
}

# Convert dictionary to Spark-compatible key-value mapping and pply
mapping_expr = create_map([lit(x) for x in chain(*frequency_mapping.items())])
silver_shopping_df = silver_shopping_df.withColumn(
    "Frequency_Purchases(per_year)",
    mapping_expr[silver_shopping_df["Frequency_of_Purchases"]]
)

In [0]:
# Show the first 12 rows
display(silver_shopping_df.limit(12))

In [0]:
# silver_shopping_df.withColumnRenamed('Purchase_Amount__USD','')

In [0]:
silver_shopping_df.withColumnRenamed('Frequency_Purchases(per_year)', 'Frequency_Purchases_per_year').write.format("delta") \
    .mode("append") \
    .option("overwriteSchema", "true") \
    .saveAsTable("brian_work.silver_layer.silver_shopping_data")

In [0]:
#Check table in Bronze Layer
spark.sql("SHOW TABLES IN brian_work.silver_layer").show()